Copyright (c) 2023 Graphcore Ltd. All rights reserved.

## Molecular property prediction on IPU using GIN - Training with PyTorch Geometric

In this tutorial we show an example of how to perform the task of graph classification using the [Graph Isomorphism Network](https://arxiv.org/pdf/1810.00826.pdf) architecture on the Graphcore IPU. This tutorial builds on the [Graph Classification](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=1tBMhOrq4JKw) introduction notebook provided by PyTorch Geometric with modifications required in order to use Poptorch. 

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------------|----------------|
|   GNNs   |  Graph Classification  | GIN | NCI1 | Training, evaluation | recommended: 4 | ~2 minutes |

This notebook assumes some familiarity with PopTorch as well as PyTorch Geometric (PyG).  For additional resources please consult:

* [PopTorch Documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/index.html)
* [PopTorch Examples and Tutorials](https://docs.graphcore.ai/en/latest/examples.html#pytorch)
* [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
* [PopTorch Geometric Documentation](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/en/latest/index.html)

### Running on Paperspace

The Paperspace environment lets you run this notebook with no set up. To improve your experience we preload datasets and pre-install packages, this can take a few minutes, if you experience errors immediately after starting a session please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).

Requirements:

* Python packages installed with `pip install -r requirements.txt`

In [ ]:
%pip install -q -r requirements.txt

And for compatibility with the Paperspace environment variables we will do the following:

In [ ]:
import os
import os.path as osp

import poptorch

poptorch.setLogLevel("ERR")
executable_cache_dir = (
    os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/pyg-gin"
)
dataset_directory = os.getenv("DATASETS_DIR", "data")

Now we are ready to start!

## NCI1 Dataset

Graph classification refers to the problem of classifiying entire graphs (in contrast to nodes), given a dataset of graphs, based on some structural graph properties. The aim is to embed entire graphs in such a way so that they are linearly separable given a task at hand.

The most common task for graph classification is molecular property prediction, in which molecules are represented as graphs, and the task may be to e.g. infer whether a molecule inhibits HIV virus replication or not.

The TU Dortmund University has collected a wide range of different graph classification datasets, known as the [TUDatasets](https://chrsmrrs.github.io/datasets/), which are accessible via [torch_geometric.datasets.TUDataset](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset) in PyTorch Geometric. Below a medium sized dataset, the NCI1 dataset.

The NCI1 dataset comes from the cheminformatics domain:
- Each graph in the dataset is molecule that has either a positive or negative associatiaion with cell lung cancer
- Each node or vertex in the molecule is an atom and has a node attribute corresponding to the atom's type, represented via a one-hot encoding scheme
- Each edge in the graph corresponds to bonds between the atoms

Lets first load the dataset:

In [ ]:
from torch_geometric.datasets import TUDataset

tudataset_root = osp.join(dataset_directory, "TUDataset")
dataset = TUDataset(root=tudataset_root, name="NCI1")

And take a look at the first item:

In [ ]:
data = dataset[0]
print(f"Graph zero summary:")
print(f"Average node degree: {data.num_edges / data.num_nodes:.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Has edge features: {data.edge_attr is not None}")
print(f"Is undirected: {data.is_undirected()}")
print(f"Molecule label: {data.y.item()}")

We can also get a summary of the entire dataset:

In [ ]:
dataset_summary = dataset.get_summary()
print(dataset_summary)

As expected the mean number of nodes and edges in the dataset is small. We will need to create items of these batches for training, in the next section we will see how to do that.

## Data Loading for the IPU   

Now that we understand the dataset, lets prepare our data ready for loading.

First lets split our dataset so we have 80% of the data for training and 20% for evaluation. 

In [ ]:
split = int(len(dataset) * 0.8)
train_dataset = dataset[split:]
test_dataset = dataset[:split]

To provide a batch of graphs to the model rather than individual molecules, we construct dataloaders for both the training and test data. As our data is many small graphs we can use packing to batch our graphs into batches of fixed size, required for the IPU. Packing will make our batches have less wasted space and therefore our computation to be more efficient. For more information on the details of packing see our [Small Graph batching using Packing tutorial](../../../tutorials/tutorials/pytorch_geometric/4_small_graph_batching_with_packing/).

To do this we need to determine the output size of our batches. First lets start with a batch size of `64` and calculate the sizes to make up the batches:

- $\operatorname{MAX}(\{\operatorname{NUMNODES}(G_i) ; i=1,\ldots,N\}) \times \text{batch_size}$
- $\operatorname{MAX}(\{\operatorname{NUMEDGES}(G_i) ; i=1,\ldots,N\}) \times \text{batch_size}$

We can use `FixedSizeOptions.from_dataset` to automatically calculate this for this dataset.

In [ ]:
from poptorch_geometric.fixed_size_options import FixedSizeOptions

num_graphs = 64
fixed_size_options = FixedSizeOptions.from_dataset(train_dataset, num_graphs)
print(fixed_size_options)

Lets create a dataloader with these. We will also include `add_masks_to_batch` so that the dataloader creates masks that will be easy for us to use in the model.

In [ ]:
from poptorch_geometric.dataloader import FixedSizeDataLoader

train_loader = FixedSizeDataLoader(
    dataset=train_dataset,
    batch_size=num_graphs,
    fixed_size_options=fixed_size_options,
    drop_last=True,
)

Lets take a look at the first two samples:

In [ ]:
train_loader_iter = iter(train_loader)
first_sample = next(train_loader_iter)
second_sample = next(train_loader_iter)
print(f"first_sample = {first_sample}")
print(f"second_sample = {second_sample}")

Now our batches are fixed size but how much of our batches are wasted space due to padding? Nodes mask can show us how many of our nodes are padded and how many are real.

In [ ]:
first_sample.nodes_mask

And in this first batch how many are real and how many are padded nodes:

In [ ]:
float(first_sample.nodes_mask.sum() / len(first_sample.nodes_mask))

That is quite a low amount of real compared to padded nodes in this batch, lets try to improve it!

By default the `FixedSizeDataLoader` uses the `PadToMax` fixed size strategy. This strategy creates mini-batches of a set number of samples and then pads each mini-batch to a fixed-size. Using this strategy enough space must be allocated to ensure that every sampled mini-batch can fit in. This can waste a lot of space in the mini-batch as we some mini-batches may contain many small samples, resulting in a large amount of padding to get the mini-batch up to fixed size. Let's try the `StreamPack` strategy which instead will pack samples into a batch up to the specified fixed-size, resulting in a varying number of samples in a mini-batch but reducing the amount of wasted space. We will also increase the maximum number of graphs in the mini-batch to `300` to allow more graphs to be packed into our mini-batches if there is still space.

In [ ]:
from poptorch_geometric import FixedSizeStrategy

fixed_size_options.num_graphs = 300

train_loader = FixedSizeDataLoader(
    dataset=train_dataset,
    batch_size=fixed_size_options.num_graphs,
    fixed_size_options=fixed_size_options,
    fixed_size_strategy=FixedSizeStrategy.StreamPack,
    drop_last=True,
)
first_sample = next(iter(train_loader))
float(first_sample.nodes_mask.sum() / len(first_sample.nodes_mask))

That's much better, our batches should be much more efficient and if you inspect the sizes they will still be the sizes we obtained before. One thing we will have to bear in mind going forward is that our batches may not all have the same number of real graphs in as padded graphs. We can see that by looking at the `graphs_mask` for each batch:

In [ ]:
for i, batch in enumerate(train_loader):
    print(f"Batch {i} has {batch.graphs_mask.sum()} real graphs")

We can also configure the above dataloader to improve IO performance. This comes free from the features supported by the PopTorch dataloader. For more information on efficient data batching/loading, please refer to the resources below:
- PopTorch documentation [Efficient data batching](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/batching.html#efficient-data-batching)
- PopTorch tutorial: [Efficient data loading](https://github.com/graphcore/tutorials/tree/sdk-release-2.5/tutorials/pytorch/tut2_efficient_data_loading)

One key thing to note is that when increasing the device iterations it is useful to also change the output mode, so that the output for each iteration is returned after each step.

We also enable caching the executable so we don't need to recompile our model if we have already.

In [ ]:
poptorch_options = poptorch.Options()
poptorch_options.deviceIterations(2)
poptorch_options.outputMode(poptorch.OutputMode.All)
poptorch_options.enableExecutableCaching(executable_cache_dir)

Then recreate our dataloader with these options:

In [ ]:
train_loader = FixedSizeDataLoader(
    dataset=train_dataset,
    batch_size=fixed_size_options.num_graphs,
    fixed_size_options=fixed_size_options,
    fixed_size_strategy=FixedSizeStrategy.StreamPack,
    drop_last=True,
    options=poptorch_options,
)

Taking a look at the first batch you can see the items are twice as big as you would expect. This is because we have chosen two device iterations. PopTorch will split this single large batch between the two iterations.

In [ ]:
next(iter(train_loader))

We have a dataloader that is ready to feed data into a model, in the next section we will take a look at the model itself.

## Graph Isomorphism Network Architecture

Generally, GNNs designed for graph classification build on top of architectures for node **classification** by adding a **readout** layer and classification layer at the end of the network:

- The readout layer is used to aggregate node embeddings to generate an embedding for each graph
- The classification layer is used to accomplish the task of graph classification

Graph node embeddings using GNN architectures are typically characterised via an _aggregation_ phase and an _update_ or _combine_ phase to give a nodes $k$-hop representation:

$$a_v^{(k)}=\operatorname{AGGREGATE}^{(k)}\left(\left\{h_u^{(k-1)}: u \in \mathcal{N}(v)\right\}\right), \quad h_v^{(k)}=\operatorname{COMBINE}^{(k)}\left(h_v^{(k-1)}, a_v^{(k)}\right)$$

The Graph Isomorphism Network specialises this scheme by defining:

$$ a_v^{(k)} := \sum_{u\in \mathcal{N}{(v)}} h_u^{(k-1)},~~ h_{v}^{(k)} := \operatorname{MLP}^{(k)}((1+\epsilon^{(k)})  \cdot h_{v}^{(k-1)} + a_v^{(k)})$$

Where $h_{v}^{(0)}$ are the initial node attributes for each node. The above specialisation is a result of wanting to represent an injective function that operates on multisets of node attributes through the use of MLPs. If the $k$-hop representation of a node is defined by an injective function, it can be asserted that any two nodes that have the same $k$-hop representation are isomorphic due to the definition of injectivity. Further, any two graphs that have the same representation as generated by such an injective function will be isomorphic. This property is well suited for the task of graph classification as it allows graphs with different structural properties to differentiated. Some GNN architectures (e.g. [GraphSage](https://arxiv.org/abs/1706.02216), [GCN](https://arxiv.org/abs/1609.02907)) do not satisfy learning injective functions on multisets and therefore have drawbacks when used for the graph classification task.

To generate graph level embeddings, GIN defines the below **readout** layer which concatenates graph representations obtained from each $k$-hop node representation:

$$G_i = \operatorname{CONCAT}(\operatorname{LINEAR}(\sum_{v ;~ v \in G_i } h_v^{(k)}) ~|~ k=0,\ldots,K)$$

Unnormalized logits for each graph can therefore be obtained by applying a subsequent $\operatorname{LINEAR}$ layer.

Below is an implementation of GIN using PyTorch Geometric and Poptorch. Some clarifying notes and differences:

- The norm used is layer normalization as it doesn't require modifications when using padding in our batches and provides similar results to batch normalization.
- The loss is required to be computed in the forward method for compatibility with PopTorch.
- The loss is aggregated on all but the final padding graph, see below for more information.
- For **readout** and **classification**, rather than concatenating all $hop-k$ graph layer representations and then applying another $\operatorname{LINEAR}$ layer to obtain classification scores, the $\operatorname{LINEAR}$ layer in the definition of $G_i$ above is chosen to output classification scores, which are then aggregated over all hops. This is equivalent to $\operatorname{CONCAT}$ followed by $\operatorname{LINEAR}$ as a composition of linear functions is linear, so the learned linear map will be the same. This matches the [original implementation](https://github.com/weihua916/powerful-gnns)

For simplicity we have put the model implementation in `model.py` and we will import it here. Feel free to look over the implementation and see the points mentioned above.

In [ ]:
from model import GIN

Now we have a good understanding of the model architecture, lets start training the model.

## Training a GIN for graph classification

Now we can train and evaluate the GIN on the IPU by using the dataloaders and GIN architecture introduced above. First we set some hyperparameters and configure all that is required in order to execute our model using [PopTorch](https://github.com/graphcore/tutorials/tree/sdk-release-3.0/tutorials/pytorch/basics).

In [ ]:
in_channels = dataset.num_node_features
hidden_channels = 32
# output hidden units dimension, one unit for each class
out_channels = 2
# number of GinConv layers
num_conv_layers = 4
# number of hidden layers used in the mlp for each GinConv layer
num_mlp_layers = 2

learning_rate = 1e-2
num_epochs = 200

Now we can create the model. We will use a PopTorch optimizer that is functionally the same but more suited to the IPU for performance and memory.

In [ ]:
model = GIN(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    num_conv_layers=num_conv_layers,
    num_mlp_layers=num_mlp_layers,
    batch_size=fixed_size_options.num_graphs,
)

model.train()
model

Now we can make it into a PopTorch model ready for training:

In [ ]:
optimizer = poptorch.optim.Adam(model.parameters(), lr=learning_rate)

poptorch_training_model = poptorch.trainingModel(
    model, optimizer=optimizer, options=poptorch_options
)

Now train the model for `num_epochs`. Note below that we decay the learning rate by a factor of 0.5 every 50 epochs. 

In [ ]:
from tqdm import tqdm

epoch_bar = tqdm(range(num_epochs))

epoch_losses = []
for epoch in epoch_bar:
    epoch_loss = 0
    for data in train_loader:
        preds, micro_batch_loss = poptorch_training_model(
            data.x,
            data.edge_index,
            data.batch,
            graphs_mask=data.graphs_mask,
            target=data.y,
        )
        epoch_loss += micro_batch_loss.mean()

    # decay learning rate every 50 epochs
    if (epoch + 1) % 50 == 0:
        learning_rate *= 0.5
        optimizer = poptorch.optim.Adam(model.parameters(), lr=learning_rate)
        poptorch_training_model.setOptimizer(optimizer)

    epoch_bar.set_description(f"Epoch {epoch} training loss: {epoch_loss:0.6f}")

    epoch_losses.append(epoch_loss)

Now training is complete we can detach from the IPU:

In [ ]:
poptorch_training_model.detachFromDevice()

Lets take a look at the loss curve:

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
ax.plot(list(range(num_epochs)), epoch_losses)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
plt.grid(True)

## Evaluation

Now we have trained our model we can evaluate it on our test dataset. First lets create a test dataloader:

In [ ]:
test_loader = FixedSizeDataLoader(
    dataset=test_dataset,
    batch_size=fixed_size_options.num_graphs,
    fixed_size_options=fixed_size_options,
    fixed_size_strategy=FixedSizeStrategy.StreamPack,
    drop_last=True,
    options=poptorch_options,
)

Similarly, we wrap the model with the PopTorch inferenceModel wrapper in order for the model to execute in evaluation mode and run on the IPU. We must take special care when calculating the accuracy, ensuring we mask out any of the padded graphs before calculating the final accuracy.

In [ ]:
model.eval()
poptorch_inference_model = poptorch.inferenceModel(model, options=poptorch_options)

total_correct = 0
num_samples = 0
for data in test_loader:
    scores = poptorch_inference_model(data.x, data.edge_index, data.batch)
    preds = scores.argmax(dim=1)
    total_correct += (
        (preds.flatten() == data.y.flatten()) * data.graphs_mask.flatten()
    ).sum()
    num_samples += data.graphs_mask.sum()

accuracy = total_correct / num_samples
print(f"Test accuracy: {accuracy}")

Finally, lets detach the inference model from the IPUs:

In [ ]:
poptorch_inference_model.detachFromDevice()

We have successfully trained a GIN model and run evaluation on IPUs using packing to speed up our training.

## Follow up

Here we have seen how to train a GIN model to predict properties of small graphs. We have used packing to make efficient use of our batches while achieving fixed size.

Next you can try:

* Can you achieve a higher by adjusting some of the hyperparameters?
* Take a look at our high performing molecular prediction example using [SchNet](../../schnet/pytorch_geometric/).